# Awareness Modeling


In [3]:
import pandas as pd
import pickle
import networkx as nx
from sklearn.decomposition import NMF, LatentDirichletAllocation

## Import the features from different pickles : 

In [97]:
lang_feature_df = pd.read_pickle('../LinkingLanguages/stand_country_dist_languages.pkl')
fligh_routes_df = pd.read_pickle('../GeoMetrics/flight_routes.pickle')
hop_distance_df = pd.read_pickle('../GeoMetrics/hop_distance.pickle')
neighbor_influence_df = pd.read_pickle('../GeoMetrics/neighbor_influence.pickle')
dist_feature_df = pd.read_pickle('../GeoMetrics/real_distance.pickle')
religion_and_more_df = pd.read_pickle('../DataEnriching/6_feature_df.pickle')

#carefull the number of coutries is not the same in every pickle.
features_df = pd.concat([lang_feature_df,
           fligh_routes_df,
           hop_distance_df,
           neighbor_influence_df,
          dist_feature_df,
          religion_and_more_df],axis=1)
# convert every Nan to a 0 
features_df.fillna(0,inplace=True)

# drop the name column : 
features_df.drop('name',axis=1,inplace=True)

features_df.index.is_unique

/Users/adrian/anaconda3/envs/ada/lib/python3.6/site-packages/ipykernel/__main__.py:19: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.


True

## Create the feature matrix

In [89]:
features_mat = features_df.as_matrix()

language_dist                                                         \
              AW         AF         AO         AI         AX         AL   
AD      4.000000  16.000000   4.000000  19.000000   3.000000   3.000000   
AE      5.000000  10.000000   5.000000   4.000000   4.000000   4.000000   
AF     17.500000   0.000000  20.000000  15.000000  13.000000   7.000000   
AG     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
AI     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
AL      9.000000   7.000000   3.000000  10.000000   2.000000   0.000000   
AM     10.250000   9.500000   3.000000   7.500000   2.000000   2.000000   
AO      4.000000  20.000000   0.000000   3.000000   1.000000   3.000000   
AQ           inf        inf        inf        inf        inf        inf   
AR      4.500000  18.000000   3.000000   3.000000   2.000000   2.000000   
AS      9.666667  15.000000   3.000000   0.000000   3.000000  10.000000   
AT     13.000000  13.000000   5.000000   4.000000   4.000000   4.000000   
AU     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
AW      0.000000  17.500000   4.000000  13.000000  12.000000   9.000000   
AX     12.000000  13.000000   1.000000   3.000000   0.000000   2.000000   
AZ     10.500000   8.666667   3.000000   3.000000   2.000000   2.000000   
BA     17.500000  12.000000  20.000000  10.666667   9.333333   7.000000   
BB     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
BD     12.000000  11.000000   3.000000   4.000000   3.000000   8.000000   
BE      0.000000  16.000000   4.000000   3.000000   3.000000   3.000000   
BF      7.500000  21.000000   4.000000   3.000000   3.000000   3.000000   
BG     11.500000  12.000000   5.000000  15.000000   4.000000   4.000000   
BH      5.000000  10.000000   5.000000   4.000000   4.000000   4.000000   
BI      7.500000  21.000000   4.000000   3.000000   3.000000   3.000000   
BJ      7.500000  21.000000   4.000000   3.000000   3.000000   3.000000   
BL      7.500000  21.000000   4.000000   3.000000   3.000000   3.000000   
BM     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
BN      3.000000        inf   5.000000   3.000000   4.000000   4.000000   
BO      4.500000  18.000000   3.000000   2.500000   2.000000   2.000000   
BR      4.000000  20.000000   0.000000   3.000000   1.000000   3.000000   
..           ...        ...        ...        ...        ...        ...   
TL      4.000000  20.000000   0.000000   3.000000   1.000000   3.000000   
TM     10.500000   0.000000   3.000000   3.000000   2.000000   2.000000   
TN      5.000000  10.000000   5.000000   4.000000   4.000000   4.000000   
TO     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
TR      3.000000   4.000000   3.000000        inf   2.000000   2.000000   
TT     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
TV     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
TW      5.000000        inf   5.000000   2.000000   4.000000   4.000000   
TZ      9.333333  15.000000   2.500000   0.000000   2.500000  10.000000   
UA     11.500000  11.000000   4.000000  14.000000   3.000000   3.000000   
UG      9.333333  15.000000   2.500000   0.000000   2.500000  10.000000   
UM     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
US     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
UY      4.500000  18.000000   3.000000   3.000000   2.000000   2.000000   
UZ     10.500000   9.750000   3.000000   3.000000   2.000000   2.000000   
VA      9.500000  13.000000   7.500000  16.000000   7.500000   4.500000   
VC     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
VE      4.500000  18.000000   3.000000   3.000000   2.000000   2.000000   
VG     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
VI     13.000000  15.000000   3.000000   0.000000   3.000000  10.000000   
VN           inf        inf 